FASE 2

- Los casos de rabia ya no seran nodos solo seran atributos de las cuadras.
- Las cuadras tendran atributos del caso de rabia y de la proximidad de los casos de rabia a X metros
- Vamos a crear 2 grafos uno para entrenamiento y validacion con los datos del 2015 hasta el 2021 y otro para test con los datos del 2022 hasta el 2024

In [60]:
pip install pyreadr pandas torch fastkml torch_geometric folium geopandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [2]:
# PUESTOS DE SALUD
df_health_posts = pd.read_csv("~/Descargas/SPATIAL_DATA_AQP/Health_centers_AQP/Puestos_de_salud_APQ_12ene2024.csv", sep=";")
print(df_health_posts.shape)
df_health_posts.head()

(83, 11)


,cluster,n_microred,microred,name,lat,long,y_proj,x_proj,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,22.0,20,MARITZA CAMPOS,C.S. MARITZA CAMPO DIAZ,-16.351375,-71.562525,-1935159.862,2835969.968,NaN,NaN,NaN
1,26.0,20,MARITZA CAMPOS,P.S. CIUDAD MUNICIPAL,-16.325474,-71.594062,-1931722.877,2832729.607,NaN,NaN,NaN
2,22.0,20,MARITZA CAMPOS,P.S. PERUARBO,-16.345310,-71.600061,-1933977.297,2831792.294,NaN,NaN,NaN
3,24.0,20,MARITZA CAMPOS,P.S. NAZARENO,-16.328171,-71.549513,-1932597.555,2837747.160,NaN,NaN,NaN
4,5.0,1,FRANCISCO BOLOGNESI,C.S. FRANCISCO BOLOGNESI,-16.354767,-71.542092,-1935814.847,2838243.050,NaN,NaN,NaN


In [3]:
# TORRENTERAS
df_water_channels = pd.read_csv("~/Descargas/SPATIAL_DATA_AQP/Water_channels_AQP/Hidrografia_AQP_18ene2024.csv", sep=";")
df_water_channels.head()

,type,ident,lat,long,y_proj,x_proj
0,Rios_Principales,r_prin_001,,,-1.516416e+00,2.076337e+07
1,Rios_Principales,r_prin_001,-16.30381616,-71.48611168,-1.930532e+06,2.845257e+06
2,Rios_Principales,r_prin_001,-16.30379005,-71.48621225,-1.930528e+06,2.845246e+06
3,Rios_Principales,r_prin_001,-16.30376392,-71.48631287,-1.930523e+06,2.845235e+06
4,Rios_Principales,r_prin_001,-16.30376396,-71.48641297,-1.930522e+06,2.845224e+06


In [4]:
#  NIVEL SOCIO-ECONOMICO
df_economic_income_1 = pd.read_csv("~/Descargas/SPATIAL_DATA_AQP/Economic_income_AQP/Economic_income_cluster/Ecom_cluster_22_10may2023.csv", sep=";")
df_economic_income_2 = pd.read_csv("~/Descargas/SPATIAL_DATA_AQP/Economic_income_AQP/Economic_income_cluster/Ecom_cluster_23_21feb2023.csv", sep=";")
df_economic_income_3 = pd.read_csv("~/Descargas/SPATIAL_DATA_AQP/Economic_income_AQP/Economic_income_cluster/Ecom_cluster_24_21feb2023.csv", sep=";")
df_economic_income_4 = pd.read_csv("~/Descargas/SPATIAL_DATA_AQP/Economic_income_AQP/Economic_income_cluster/Ecom_cluster_25_21feb2023.csv", sep=";")
df_economic_income_5 = pd.read_csv("~/Descargas/SPATIAL_DATA_AQP/Economic_income_AQP/Economic_income_cluster/Ecom_cluster_26_21feb2023.csv", sep=";")
df_economic_income_6 = pd.read_csv("~/Descargas/SPATIAL_DATA_AQP/Economic_income_AQP/Economic_income_cluster/Ecom_cluster_27_21feb2023.csv", sep=";")
df_economic_income = pd.concat([df_economic_income_1, df_economic_income_2, df_economic_income_3, df_economic_income_4, df_economic_income_5, df_economic_income_6], ignore_index=True)
df_economic_income.head()

,ident,lat,long,y_proj,x_proj,ingreso_economico
0,1.4.54-1,,,-1.516416e+00,2.076337e+07,B
1,1.4.54-1,-16.3619287,-71.56312249,-1.936392e+06,2.835765e+06,B
2,1.4.54-1,-16.36042421,-71.56334575,-1.936212e+06,2.835759e+06,B
3,1.4.54-1,-16.36114067,-71.56646806,-1.936257e+06,2.835396e+06,B
4,1.4.54-1,-16.36256118,-71.5660744,-1.936429e+06,2.835422e+06,B


In [5]:
# RABIA
import os
import xml.etree.ElementTree as ET
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# Ruta a la carpeta con archivos KML
folder_path = "/home/pantro/Descargas/KML_casos_rabia/"
kml_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".kml")])

# Namespace usado en KML
ns = {"kml": "http://www.opengis.net/kml/2.2"}

features = []

for filename in kml_files:
    filepath = os.path.join(folder_path, filename)
    try:
        year = int(filename.replace(".kml", ""))
    except ValueError:
        continue  # omite archivos mal nombrados

    tree = ET.parse(filepath)
    root = tree.getroot()

    placemarks = root.findall(".//kml:Placemark", ns)
    for pm in placemarks:
        coord_tag = pm.find(".//kml:coordinates", ns)
        name_tag = pm.find("kml:name", ns)
        if coord_tag is not None:
            coords_text = coord_tag.text.strip()
            try:
                lon, lat, *_ = map(float, coords_text.split(','))
                name = name_tag.text.strip() if name_tag is not None else None
                features.append({
                    "name": name,
                    "anio": year,
                    "geometry": Point(lon, lat),
                })
            except ValueError:
                continue

# Crear GeoDataFrame
gdf_rabies = gpd.GeoDataFrame(features, crs="EPSG:4326")

# Mostrar
print(gdf_rabies.shape)
gdf_rabies.head()

# Si quieres guardar como GeoJSON
# gdf.to_file("casos_rabia_completo.geojson", driver="GeoJSON")

(398, 3)


,name,anio,geometry
0,53,2015,POINT (-71.51186 -16.36774)
1,73,2015,POINT (-71.50151 -16.37566)
2,92,2015,POINT (-71.50128 -16.39534)
3,101,2015,POINT (-71.52681 -16.41053)
4,103,2015,POINT (-71.51339 -16.40597)


## Preprosesamiento

In [6]:
# build_zamacola_graph.py
import pandas as pd
import geopandas as gpd
import torch
from torch_geometric.data import HeteroData
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon, LineString

In [7]:
# NIVEL SOCIECONOMICO
# Eliminar filas vacías (sin coordenadas)
df_economic_income_clean = df_economic_income[
    (df_economic_income["lat"].str.strip() != '') &
    (df_economic_income["long"].str.strip() != '')
].copy()

# Agrupar por 'ident' (ID de cuadra) y construir polígonos
cuadra_polys = df_economic_income_clean.groupby("ident").apply(
    lambda g: pd.Series({
        "geometry": Polygon(zip(g["long"], g["lat"])),
        "ingreso_economico": g["ingreso_economico"].unique()[0]  # único valor
    })
).reset_index()
cuadra_polys.columns = ["cuadra_id", "geometry", "ingreso_economico"]

# Crear GeoDataFrame con polígonos
gdf_economic_income = gpd.GeoDataFrame(cuadra_polys, geometry="geometry", crs="EPSG:4326")
print(gdf_economic_income.shape)
gdf_economic_income.head()

(4296, 3)


/tmp/ipykernel_128035/2957687187.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cuadra_polys = df_economic_income_clean.groupby("ident").apply(


,cuadra_id,geometry,ingreso_economico
0,1.4.100-1,"POLYGON ((-71.58095 -16.35108, -71.58178 -16.3...",D
1,1.4.100-10,"POLYGON ((-71.58352 -16.34361, -71.58349 -16.3...",D
2,1.4.100-11,"POLYGON ((-71.58344 -16.34353, -71.58179 -16.3...",D
3,1.4.100-12,"POLYGON ((-71.58582 -16.34319, -71.58592 -16.3...",D
4,1.4.100-2,"POLYGON ((-71.58400 -16.34949, -71.58416 -16.3...",D


In [8]:
# PUESTOS DE SALUD
# Suponiendo que df_health_posts tiene columnas 'lat' y 'long'
gdf_health_posts = gpd.GeoDataFrame(
    df_health_posts,
    geometry=gpd.points_from_xy(df_health_posts['long'], df_health_posts['lat']),
    crs="EPSG:4326"
)
# Filtrar filas: asegurarse que 'cluster' sea un número (ignorar NaNs o strings)
gdf_health_posts = gdf_health_posts[pd.to_numeric(gdf_health_posts['cluster'], errors='coerce').notnull()]

# Convertir 'cluster' a numérico si aún no lo es
gdf_health_posts['cluster'] = gdf_health_posts['cluster'].astype(int)

# Seleccionar solo las columnas deseadas
gdf_health_posts = gdf_health_posts[['cluster', 'name', 'lat', 'long', 'geometry']]
print(gdf_health_posts.shape)
gdf_health_posts.head()

(77, 5)


,cluster,name,lat,long,geometry
0,22,C.S. MARITZA CAMPO DIAZ,-16.351375,-71.562525,POINT (-71.56252 -16.35137)
1,26,P.S. CIUDAD MUNICIPAL,-16.325474,-71.594062,POINT (-71.59406 -16.32547)
2,22,P.S. PERUARBO,-16.345310,-71.600061,POINT (-71.60006 -16.34531)
3,24,P.S. NAZARENO,-16.328171,-71.549513,POINT (-71.54951 -16.32817)
4,5,C.S. FRANCISCO BOLOGNESI,-16.354767,-71.542092,POINT (-71.54209 -16.35477)


In [9]:
# TORRENTERAS
# Eliminar filas vacías (sin coordenadas)
df_water_channels_clean = df_water_channels[
    (df_water_channels["lat"].str.strip() != '') &
    (df_water_channels["long"].str.strip() != '')
]
# Crear geometrías LineString por grupo
lineas = (
    df_water_channels_clean.groupby("ident")
    .apply(lambda group: LineString(zip(group["long"], group["lat"])))
    .reset_index(name="geometry")
)

# Convertir a GeoDataFrame
gdf_water_channels = gpd.GeoDataFrame(lineas, geometry="geometry", crs="EPSG:4326")
print(gdf_water_channels.shape)
gdf_water_channels.head()

/tmp/ipykernel_128035/627060655.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: LineString(zip(group["long"], group["lat"])))


(218, 2)


,ident,geometry
0,r_prin_001,"LINESTRING (-71.48611 -16.30382, -71.48621 -16..."
1,r_prin_002,"LINESTRING (-71.43686 -16.42426, -71.43687 -16..."
2,r_prin_003,"LINESTRING (-71.41278 -16.57048, -71.41280 -16..."
3,r_prin_004,"LINESTRING (-71.43694 -16.49497, -71.43696 -16..."
4,torr_001,"LINESTRING (-71.46112 -16.36777, -71.46113 -16..."


## Solo los puestos de salud y los casos de rabia que estan dentro del cluster

In [70]:
# Filtrar puestos de salud y casos de rabia solo para el cluster 27
df_cluster_1 = pd.read_csv("/home/pantro/Descargas/SPATIAL_DATA_AQP/Clusters_AQP/Limites_Cluster_22_09may2025.csv", sep=";")
df_cluster_2 = pd.read_csv("/home/pantro/Descargas/SPATIAL_DATA_AQP/Clusters_AQP/Limites_Cluster_23_09may2025.csv", sep=";")
df_cluster_3 = pd.read_csv("/home/pantro/Descargas/SPATIAL_DATA_AQP/Clusters_AQP/Limites_Cluster_24_09may2025.csv", sep=";")
df_cluster_4 = pd.read_csv("/home/pantro/Descargas/SPATIAL_DATA_AQP/Clusters_AQP/Limites_Cluster_25_09may2025.csv", sep=";")
df_cluster_5 = pd.read_csv("/home/pantro/Descargas/SPATIAL_DATA_AQP/Clusters_AQP/Limites_Cluster_26_09may2025.csv", sep=";")
df_cluster_6 = pd.read_csv("/home/pantro/Descargas/SPATIAL_DATA_AQP/Clusters_AQP/Limites_Cluster_27_09may2025.csv", sep=";")
# Unir los DataFrames
df_cluster = pd.concat([df_cluster_1, df_cluster_2, df_cluster_3, df_cluster_4, df_cluster_5, df_cluster_6], ignore_index=True)
# 1. Eliminar filas sin coordenadas (NaN o vacías)
df_cluster_clean = df_cluster[
    (df_cluster["lat"].str.strip() != '') &
    (df_cluster["long"].str.strip() != '')
]

# 2. Asegurar que las coordenadas sean float
df_cluster_clean["lat"] = df_cluster_clean["lat"].astype(float)
df_cluster_clean["long"] = df_cluster_clean["long"].astype(float)

# 3. Agrupar por 'ident' y construir polígonos
cluster_polys = df_cluster_clean.groupby("ident").apply(
    lambda g: Polygon(zip(g["long"], g["lat"]))
).reset_index(name="geometry")

# 4. Crear GeoDataFrame
gdf_cluster = gpd.GeoDataFrame(cluster_polys, geometry="geometry", crs="EPSG:4326")
print(gdf_cluster.head())

# OBTENER SOLO LOS PUESTOS DE SALUD DENTRO DEL CLUSTER
gdf_health_posts = gpd.sjoin(
    gdf_health_posts,
    gdf_cluster,
    how="inner",
    predicate="intersects"
)
print("---------------------------------------")
print(gdf_health_posts.shape)
print(gdf_health_posts.head())

print("=======================================================")
# OBTENER SOLO LOS CASOS DE RABIA DENTRO DEL CLUSTER
gdf_rabies = gpd.sjoin(
    gdf_rabies,
    gdf_cluster,
    how="inner",
    predicate="intersects"
)
print("---------------------------------------")
print(gdf_rabies.shape)
print(gdf_rabies.head())


/tmp/ipykernel_86760/412664487.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster_clean["lat"] = df_cluster_clean["lat"].astype(float)
/tmp/ipykernel_86760/412664487.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cluster_clean["long"] = df_cluster_clean["long"].astype(float)
/tmp/ipykernel_86760/412664487.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be exclude

         ident                                           geometry
0  ZAMACOLA_01  POLYGON ((-71.54220 -16.32096, -71.54209 -16.3...
1  ZAMACOLA_02  POLYGON ((-71.56913 -16.32129, -71.56928 -16.3...
2  ZAMACOLA_03  POLYGON ((-71.57338 -16.32124, -71.57415 -16.3...
3  ZAMACOLA_04  POLYGON ((-71.60649 -16.32654, -71.60492 -16.3...
4  ZAMACOLA_05  POLYGON ((-71.61663 -16.32034, -71.61677 -16.3...
---------------------------------------
(4, 7)
   cluster                     name        lat       long  \
0       22  C.S. MARITZA CAMPO DIAZ -16.351375 -71.562525   
1       26    P.S. CIUDAD MUNICIPAL -16.325474 -71.594062   
2       22            P.S. PERUARBO -16.345310 -71.600061   
3       24            P.S. NAZARENO -16.328171 -71.549513   

                      geometry  index_right        ident  
0  POINT (-71.56252 -16.35137)            5  ZAMACOLA_06  
1  POINT (-71.59406 -16.32547)            3  ZAMACOLA_04  
2  POINT (-71.60006 -16.34531)            5  ZAMACOLA_06  
3  POINT (-71.5

In [71]:
import folium
from folium import FeatureGroup, LayerControl, Marker, PolyLine, GeoJson
import geopandas as gpd

# Mapa base centrado en Arequipa
m = folium.Map(location=[-16.3989, -71.5350], zoom_start=12)

# === CAPA 1: Casos de RABIA (Points) ===
fg_rabies = FeatureGroup(name="Casos de Rabia")
for _, row in gdf_rabies.iterrows():
    Marker(
        location=[row.geometry.y, row.geometry.x],
        popup="Caso de rabia",
        icon=folium.Icon(color="red", icon="exclamation-sign")
    ).add_to(fg_rabies)
fg_rabies.add_to(m)

# === CAPA 2: Nivel Socioeconómico (Polygons) ===
fg_income = FeatureGroup(name="Nivel Socioeconómico")
GeoJson(
    gdf_economic_income,
    name="Nivel Socioeconómico",
    style_function=lambda x: {
        'fillColor': '#00b894',
        'color': '#2d3436',
        'weight': 1,
        'fillOpacity': 0.4,
    },
    tooltip=folium.GeoJsonTooltip(fields=["cuadra_id"])
).add_to(fg_income)
fg_income.add_to(m)

# === CAPA 3: Puestos de Salud (Points) ===
fg_salud = FeatureGroup(name="Puestos de Salud")
for _, row in gdf_health_posts.iterrows():
    Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row["name"],
        icon=folium.Icon(color="blue", icon="plus-sign")
    ).add_to(fg_salud)
fg_salud.add_to(m)

# === CAPA 4: Torrenteras (LineStrings) ===
fg_torrenteras = FeatureGroup(name="Torrenteras")
GeoJson(
    gdf_water_channels,
    name="Torrenteras",
    style_function=lambda x: {
        'color': '#6c5ce7',
        'weight': 2,
    },
    tooltip=folium.GeoJsonTooltip(fields=["ident"])
).add_to(fg_torrenteras)
fg_torrenteras.add_to(m)

# === CAPA 5: Clústeres (Polygons) ===
fg_cluster = FeatureGroup(name="Clústeres")
GeoJson(
    gdf_cluster,
    name="Clústeres",
    style_function=lambda x: {
        'fillColor': '#ffeaa7',
        'color': '#d35400',
        'weight': 2,
        'fillOpacity': 0.3,
    },
    tooltip=folium.GeoJsonTooltip(fields=["ident"])
).add_to(fg_cluster)
fg_cluster.add_to(m)

# === Control de capas ===
folium.LayerControl(collapsed=False).add_to(m)

# === Guardar o mostrar ===
m.save("mapa_solo_objetivo_fase2.html")


In [72]:
#- SEPARAR CASOS DE RABIA DEL 2015-2021 PARA TRAIN_VAL Y 2022-2024 PARA TEST
gdf_rabies_train_val = gdf_rabies[(gdf_rabies["anio"] > 2015) & (gdf_rabies["anio"] < 2022)]
gdf_rabies_test = gdf_rabies[(gdf_rabies["anio"] > 2021)]
print("Cantidad de registros 2016-2021: ",len(gdf_rabies_train_val))
print(gdf_rabies_train_val["anio"].value_counts())
print("Cantidad de registros 2022-2024: ",len(gdf_rabies_test))
print(gdf_rabies_test["anio"].value_counts())

Cantidad de registros 2016-2021:  145
anio
2021    44
2016    32
2018    23
2017    22
2019    17
2020     7
Name: count, dtype: int64
Cantidad de registros 2022-2024:  16
anio
2022    11
2023     3
2024     2
Name: count, dtype: int64


# Generar el grafo para TRAIN_VAL

In [127]:
# build_zamacola_graph.py (versión con parámetros configurables y visualización)

import pandas as pd
import geopandas as gpd
import torch
from torch_geometric.data import HeteroData
from shapely.geometry import Point
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# TEST  # TRAIN_VAL
gdf_rabies = gdf_rabies_test.copy()#gdf_rabies_train_val.copy()

# Cambiar a "EPSG:32719"
gdf_rabies_proj = gdf_rabies.to_crs("EPSG:32719")
gdf_economic_income_proj = gdf_economic_income.to_crs("EPSG:32719")
gdf_health_posts_proj = gdf_health_posts.to_crs("EPSG:32719")
gdf_water_channels_proj = gdf_water_channels.to_crs("EPSG:32719")

# ---------------------------
# PARÁMETROS CONFIGURABLES
# ---------------------------
CUADRA_NEIGHBOR_RADIUS = 200  # metros para definir vecindad entre cuadras
CUADRA_TORRENTERA_DIST = 500  # metros
CUADRA_PROXIMITY_RABIES_CASE_RADIUS = 100  # metros. Se busca a las cuadras que esten dentro de 100 metros para decir que todas ellas tienen rabia

# ---------------------------
# CARGA DE DATOS (GeoDataFrames preprocesados):
# gdf_economic_income: cuadras como POLYGON
# gdf_health_posts: centros de salud como POINT
# gdf_water_channels: torrenteras como LINESTRING
# gdf_rabies: casos de rabia como POINT + año
# ---------------------------

# Asegurar proyección métrica
cuadras = gdf_economic_income_proj.copy()
rabies = gdf_rabies_proj.copy()

# -----------------------------
# Calcular centroide + coordenadas
# -----------------------------
cuadras["centroid"] = cuadras.geometry.centroid
cuadras["lat"] = cuadras.centroid.y
cuadras["long"] = cuadras.centroid.x

# -----------------------------
# One-hot encoding ingreso económico
# -----------------------------
cuadras = pd.get_dummies(cuadras, columns=["ingreso_economico"], prefix="ingreso")

# -----------------------------
# Casos de rabia dentro de la cuadra
# -----------------------------
cuadras["num_rabies_cases_in_cuadra"] = cuadras.geometry.apply(
    lambda poly: rabies[rabies.within(poly)].shape[0]
)
# Contar cantidad de cuadras con al menos un caso de rabia
num_cuadras_afectadas = (cuadras["num_rabies_cases_in_cuadra"] > 0).sum()
print(f"Cantidad de cuadras afectadas: {num_cuadras_afectadas}")


# -----------------------------
# Distancia al caso de rabia más cercano en 100 m
# -----------------------------
def min_dist_to_case(poly_centroid, rabies_points, max_dist):
    distances = rabies_points.distance(poly_centroid)
    close = distances[distances <= max_dist]
    return close.min() if not close.empty else np.nan

cuadras["min_dist_to_rabies_case"] = cuadras["centroid"].apply(
    lambda p: min_dist_to_case(p, rabies.geometry, CUADRA_PROXIMITY_RABIES_CASE_RADIUS)
)
# Rellenar NaN con un valor alto para que redes neuronales lo manejen mejor
cuadras["min_dist_to_rabies_case"] = cuadras["min_dist_to_rabies_case"].fillna(9999)

# -----------------------------
# Selección de columnas de features
# -----------------------------
feature_columns = (
    ["lat", "long", "num_rabies_cases_in_cuadra", "min_dist_to_rabies_case"]
    #["lat", "long", "min_dist_to_rabies_case"] # Teste para la fase2_1
    + [c for c in cuadras.columns if c.startswith("ingreso_")]
)

# -----------------------------
# Normalizar features
# -----------------------------
scaler = MinMaxScaler()
cuadra_features = scaler.fit_transform(cuadras[feature_columns])

Cantidad de cuadras afectadas: 13


In [119]:
#cuadras_filtradas = cuadras[cuadras["num_rabies_cases_in_cuadra"] > 1]
#cuadras_filtradas.head()

,cuadra_id,geometry,centroid,lat,long,ingreso_B,ingreso_C,ingreso_D,ingreso_E,num_rabies_cases_in_cuadra,min_dist_to_rabies_case


In [128]:
# ---------------------------
# CREAR GRAFO HETEROGÉNEO
# ---------------------------

data = HeteroData()

# Cuadras como nodos
data["cuadra"].x = torch.tensor(cuadra_features, dtype=torch.float)
data['cuadra'].feature_columns = feature_columns
data["cuadra"].cuadra_id = cuadras["cuadra_id"].tolist()
data["cuadra"].node_index = cuadras.index.to_list()  # <-- Aquí guardar índices reales

## OJO: Solo por esta vez, porque esto debe de manipularse en el modelo
# Crear y guardar etiquetas binarias directamente
#col_idx = feature_columns.index("num_rabies_cases_in_cuadra")
#num_cases_tensor = data['cuadra'].x[:, col_idx]
#data['cuadra'].y = (num_cases_tensor > 0).long()
data['cuadra'].y = torch.tensor(
    (cuadras["num_rabies_cases_in_cuadra"] > 0).astype(int).values,
    dtype=torch.long
)
num_pos = torch.sum(data['cuadra'].y).item()
num_total = data['cuadra'].y.size(0)
num_neg = num_total - num_pos
print(f"Positivos (1): {num_pos}")
print(f"Negativos (0): {num_neg}")

# Puestos de salud como nodos
health_posts = gdf_health_posts_proj.copy()
data["health_post"].x = torch.tensor(health_posts[["lat", "long"]].values, dtype=torch.float)
data["health_post"].name = health_posts["name"].tolist()
data["health_post"].node_index = health_posts.index.to_list()  # <-- guardar índices también

# Torrenteras como nodos
torrenteras = gdf_water_channels_proj.copy()
torrenteras["lat"] = torrenteras.geometry.centroid.y
torrenteras["long"] = torrenteras.geometry.centroid.x
data["water_channel"].x = torch.tensor(torrenteras[["lat", "long"]].values, dtype=torch.float)
data["water_channel"].ident = torrenteras["ident"].tolist()
data["water_channel"].node_index = torrenteras.index.to_list()  # <-- guardar índices

# 🦠 Casos de rabia como nodos con año + punto geográfico
#gdf_rabies_proj = gdf_rabies_proj.reset_index(drop=True)
#gdf_rabies_proj["lat"] = gdf_rabies_proj.geometry.y
#gdf_rabies_proj["long"] = gdf_rabies_proj.geometry.x
#data["rabies_case"].x = torch.tensor(gdf_rabies_proj[["lat", "long"]].values, dtype=torch.float)
#data["rabies_case"].name = gdf_rabies_proj["name"].tolist()
#data["rabies_case"].year = torch.tensor(gdf_rabies_proj["anio"].values, dtype=torch.long)
#data["rabies_case"].node_index = gdf_rabies_proj.index.to_list()  # <-- guardar índices

Positivos (1): 13
Negativos (0): 4283


In [129]:
# COMPROBACION DE DATOS
## PARA PROPOSITOS DE VISUALIZACION Y VER QUE TODO VAYA BIEN
#

print("------------------------------------------------------------------")
print("*****************  df_preview_cuadra  *************************")
print("------------------------------------------------------------------")
df_preview_cuadra = pd.DataFrame({
    "cuadra_id": data["cuadra"].cuadra_id[:5],
    "node_index": data["cuadra"].node_index[:5],
    "lat": data["cuadra"].x[:5, 0].tolist(),
    "long": data["cuadra"].x[:5, 1].tolist()
})

print(df_preview_cuadra)

print("------------------------------------------------------------------")
print("*****************  df_preview_health_post  *************************")
print("------------------------------------------------------------------")
df_preview_health_post = pd.DataFrame({
    "name": data["health_post"].name[:5],
    "node_index": data["health_post"].node_index[:5],
    "lat": data["health_post"].x[:5, 0].tolist(),
    "long": data["health_post"].x[:5, 1].tolist()
})

print(df_preview_health_post)

------------------------------------------------------------------
*****************  df_preview_cuadra  *************************
------------------------------------------------------------------
    cuadra_id  node_index       lat      long
0   1.4.100-1           0  0.343036  0.524072
1  1.4.100-10           1  0.401710  0.511256
2  1.4.100-11           2  0.406504  0.519437
3  1.4.100-12           3  0.405803  0.493044
4   1.4.100-2           4  0.341268  0.507758
------------------------------------------------------------------
*****************  df_preview_health_post  *************************
------------------------------------------------------------------
                      name  node_index        lat       long
0  C.S. MARITZA CAMPO DIAZ           0 -16.351376 -71.562523
1    P.S. CIUDAD MUNICIPAL           1 -16.325474 -71.594063
2            P.S. PERUARBO           2 -16.345310 -71.600060
3            P.S. NAZARENO           3 -16.328171 -71.549515


In [130]:
# ---------------------------
# RELACIONES (ARISTAS)
# ---------------------------

# ---------------------------
# Cuadra ↔ Cuadra (vecinas por distancia entre centroides)
# ---------------------------
# Conectamos centroides de cuadras si están a menos de CUADRA_NEIGHBOR_RADIUS (100m)

# Extraer coordenadas de los centroides
cuadra_coords = cuadras[["cuadra_id", "long", "lat"]].values

# Generar relaciones de vecindad
edges_src, edges_dst = [], []
for i in range(len(cuadra_coords)):
    for j in range(i + 1, len(cuadra_coords)):
        _, x1, y1 = cuadra_coords[i]
        _, x2, y2 = cuadra_coords[j]
        dist = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5  # Distancia euclidiana
        if dist < CUADRA_NEIGHBOR_RADIUS:
            edges_src.append(i)
            edges_dst.append(j)

# Hacer grafo no dirigido (duplicar aristas en sentido contrario)
edges_src += edges_dst
edges_dst += edges_src[:len(edges_dst)]

# Convertir a tensor de aristas
if edges_src:
    data["cuadra", "vecina_de", "cuadra"].edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long)

In [131]:
# ---------------------------
# Cuadra → Puesto de salud más cercano y pone la distancia como atributo
# ---------------------------
# Asegurar que las coordenadas estén listas
cuadras["x"] = cuadras["centroid"].x
cuadras["y"] = cuadras["centroid"].y

health_posts["x"] = health_posts.geometry.x
health_posts["y"] = health_posts.geometry.y

# Crear relaciones
edges_cuadra_ps = []
distancias = []

for i, cuadra in cuadras.iterrows():
    dists = health_posts.apply(
        lambda row: ((cuadra.x - row.x) ** 2 + (cuadra.y - row.y) ** 2) ** 0.5, axis=1
    )
    closest = dists.idxmin()
    distance = dists[closest]
    edges_cuadra_ps.append((i, closest))
    distancias.append(distance)

# Crear edge_index
src, dst = zip(*edges_cuadra_ps)
data["cuadra", "reporta_a", "health_post"].edge_index = torch.tensor([src, dst], dtype=torch.long)

# Normalizar distancias
scaler = MinMaxScaler()
edge_attr_np = scaler.fit_transform(torch.tensor(distancias, dtype=torch.float).reshape(-1, 1))
edge_attr = torch.tensor(edge_attr_np, dtype=torch.float)

# Asignar atributo (relación original)
data["cuadra", "reporta_a", "health_post"].edge_attr = edge_attr

# 🔁 Agregar relación inversa: health_post → cuadra
data["health_post", "atiende_a", "cuadra"].edge_index = torch.tensor([dst, src], dtype=torch.long)
data["health_post", "atiende_a", "cuadra"].edge_attr = edge_attr  # o edge_attr.clone() si vas a modificar

In [132]:
# ---------------------------
# Cuadra → Torrenteras (con distancia como atributo)
# ---------------------------
edges_cuadra_torr = []
distancias_torr = []

for i, cuadra in cuadras.iterrows():
    for j, torr in torrenteras.iterrows():
        dist = cuadra.geometry.distance(torr.geometry)  # distancia mas corta entre los elemento en metros
        if dist < CUADRA_TORRENTERA_DIST:
            edges_cuadra_torr.append((i, j))
            distancias_torr.append(dist)

if edges_cuadra_torr:
    src, dst = zip(*edges_cuadra_torr)
    data["cuadra", "cerca_de", "water_channel"].edge_index = torch.tensor([src, dst], dtype=torch.long)

    # Normalizar y añadir edge_attr
    scaler = MinMaxScaler()
    edge_attr_torr = scaler.fit_transform(torch.tensor(distancias_torr).reshape(-1, 1))
    data["cuadra", "cerca_de", "water_channel"].edge_attr = torch.tensor(edge_attr_torr, dtype=torch.float)


In [79]:
'''
# ---------------------------
# 🦠 Caso de rabia → Cuadra (sjoin espacial)
# ---------------------------

# Eliminar columnas conflictivas si existen
gdf_rabies_proj = gdf_rabies_proj.drop(columns=["index_left", "index_right"], errors="ignore")
cuadras = cuadras.drop(columns=["index_left", "index_right"], errors="ignore")

# Reparar geometrías inválidas
cuadras["geometry"] = cuadras["geometry"].buffer(0)

# Unificar CRS
gdf_rabies_proj = gdf_rabies_proj.to_crs(cuadras.crs)

# Spatial join (casos de rabia dentro de alguna cuadra)
joined = gpd.sjoin(gdf_rabies_proj, cuadras, how="left", predicate="intersects")

# Mapeos de índice original a índice relativo
rabies_id_to_rel = {nid: i for i, nid in enumerate(data["rabies_case"].node_index)}
cuadra_id_to_rel = {nid: i for i, nid in enumerate(data["cuadra"].node_index)}

# Aristas caso de rabia → cuadra
edges_rabia_cuadra = [] # Casos de rabia dentros de la cuadra
edges_riesgo_proximidad = [] # Casos de rabia fuera de las cuadras
# Arista cuadra -> caso de rabia
edges_cercano_a_caso = []

# |= Casos que cayeron dentro de una cuadra
for _, row in joined.dropna(subset=["index_right"]).iterrows():
    src_orig = row.name
    dst_orig = int(row["index_right"])
    if src_orig in rabies_id_to_rel and dst_orig in cuadra_id_to_rel:
        src = rabies_id_to_rel[src_orig]
        dst = cuadra_id_to_rel[dst_orig]
        edges_rabia_cuadra.append((src, dst))

# Construyendo la relacion
if edges_rabia_cuadra:
    src, dst = zip(*edges_rabia_cuadra)
    data["rabies_case", "ocurre_en", "cuadra"].edge_index = torch.tensor([src, dst], dtype=torch.long)# Convertir a tensor de aristas
# Crear aristas inversas
edge_index_ocurre_en = data["rabies_case", "ocurre_en", "cuadra"].edge_index
data["cuadra", "tiene_caso", "rabies_case"].edge_index = edge_index_ocurre_en.flip(0)


# |= Propagacion de los casos de rabia a las cuadras dentro del radio de RABIES_CASE_WITHOUT_CUADRA (100m)
no_match = joined.copy()
cuadras_afectadas = set()
casos_dentro_cuadra = joined["index_right"].notna().sum()
casos_fuera_cuadra = joined["index_right"].isna().sum()

for _, row in no_match.iterrows():
    src_orig = row.name
    caso_point = row.geometry
    dists = cuadras.geometry.distance(caso_point)
    
    # Filtrar las cuadras dentro del rango definido
    cuadras_cercanas = dists[dists <= RABIES_CASE_PROXIMITY_CUADRA]

    for dst_orig in cuadras_cercanas.index:
        if src_orig in rabies_id_to_rel and dst_orig in cuadra_id_to_rel:
            src = rabies_id_to_rel[src_orig]
            dst = cuadra_id_to_rel[dst_orig]
            
            # Relación existente
            edges_rabia_cuadra.append((src, dst))
            cuadras_afectadas.add(dst_orig)# Guardar para contar luego
            
            # Nuevas relaciones
            edges_riesgo_proximidad.append((src, dst))  # rabies_case → cuadra
            edges_cercano_a_caso.append((dst, src))     # cuadra → rabies_case

# Crear nuevas relaciones si existen
if edges_riesgo_proximidad:
    src, dst = zip(*edges_riesgo_proximidad)
    data["rabies_case", "riesgo_por_proximidad", "cuadra"].edge_index = torch.tensor([src, dst], dtype=torch.long)

if edges_cercano_a_caso:
    src, dst = zip(*edges_cercano_a_caso)
    data["cuadra", "cercano_a_caso", "rabies_case"].edge_index = torch.tensor([src, dst], dtype=torch.long)

#-------------- Mostrar en pantalla --------------------------
print(f"✔️ Casos de rabia dentro de una cuadra: {casos_dentro_cuadra}")
print(f"❌ Casos de rabia fuera de una cuadra: {casos_fuera_cuadra}")
print(f"🏘️ Cuadras afectadas por 'riesgo_por_proximidad': {len(cuadras_afectadas)}")
'''


'\n# ---------------------------\n# 🦠 Caso de rabia → Cuadra (sjoin espacial)\n# ---------------------------\n\n# Eliminar columnas conflictivas si existen\ngdf_rabies_proj = gdf_rabies_proj.drop(columns=["index_left", "index_right"], errors="ignore")\ncuadras = cuadras.drop(columns=["index_left", "index_right"], errors="ignore")\n\n# Reparar geometrías inválidas\ncuadras["geometry"] = cuadras["geometry"].buffer(0)\n\n# Unificar CRS\ngdf_rabies_proj = gdf_rabies_proj.to_crs(cuadras.crs)\n\n# Spatial join (casos de rabia dentro de alguna cuadra)\njoined = gpd.sjoin(gdf_rabies_proj, cuadras, how="left", predicate="intersects")\n\n# Mapeos de índice original a índice relativo\nrabies_id_to_rel = {nid: i for i, nid in enumerate(data["rabies_case"].node_index)}\ncuadra_id_to_rel = {nid: i for i, nid in enumerate(data["cuadra"].node_index)}\n\n# Aristas caso de rabia → cuadra\nedges_rabia_cuadra = [] # Casos de rabia dentros de la cuadra\nedges_riesgo_proximidad = [] # Casos de rabia fuera d

In [133]:
# PARA VERIFICAR LA CANTIDAD DE NODOS Y ARISTAS
#print("Nodos en 'rabies_case':", data["rabies_case"].num_nodes)
print("Nodos en 'cuadra':", data["cuadra"].num_nodes)
print("Relaciones y número de aristas:")
for etype in data.edge_types:
    edge_storage = data[etype]
    print(f"{etype}: num_edges={getattr(edge_storage, 'num_edges', 'unknown')}")

Nodos en 'cuadra': 4296
Relaciones y número de aristas:
('cuadra', 'vecina_de', 'cuadra'): num_edges=64086
('cuadra', 'reporta_a', 'health_post'): num_edges=4296
('health_post', 'atiende_a', 'cuadra'): num_edges=4296
('cuadra', 'cerca_de', 'water_channel'): num_edges=9228


In [134]:
# ---------------------------
# GUARDAR EL GRAFO
# ---------------------------
torch.save(data, "/home/pantro/Documentos/GITHUB/rabies-graph-neural-model/test_zamacola_heterograph_fase2.pt")
print("✅ Grafo heterogéneo guardado")

✅ Grafo heterogéneo guardado


In [82]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


## Cargando el grafo guardado

In [83]:
import torch

# Cargar el grafo
data = torch.load("/home/pantro/Documentos/GITHUB/rabies-graph-neural-model/zamacola_heterograph_fase2.pt", weights_only=False)

print("✅ Grafo heterogéneo cargado correctamente")
print(isinstance(data, HeteroData))  # Debería imprimir: True
print(data)

✅ Grafo heterogéneo cargado correctamente
True
HeteroData(
  cuadra={
    x=[4296, 6],
    cuadra_id=[4296],
    node_index=[4296],
  },
  health_post={
    x=[4, 2],
    name=[4],
    node_index=[4],
  },
  water_channel={
    x=[218, 2],
    ident=[218],
    node_index=[218],
  },
  rabies_case={
    x=[161, 2],
    name=[161],
    year=[161],
    node_index=[161],
  },
  (cuadra, vecina_de, cuadra)={ edge_index=[2, 64086] },
  (cuadra, reporta_a, health_post)={
    edge_index=[2, 4296],
    edge_attr=[4296, 1],
  },
  (health_post, atiende_a, cuadra)={
    edge_index=[2, 4296],
    edge_attr=[4296, 1],
  },
  (cuadra, cerca_de, water_channel)={
    edge_index=[2, 9228],
    edge_attr=[9228, 1],
  },
  (rabies_case, ocurre_en, cuadra)={ edge_index=[2, 91] },
  (cuadra, tiene_caso, rabies_case)={ edge_index=[2, 91] },
  (rabies_case, riesgo_por_proximidad, cuadra)={ edge_index=[2, 1237] },
  (cuadra, cercano_a_caso, rabies_case)={ edge_index=[2, 1237] }
)


# Guardar en formato para Neo4j

In [135]:
import torch
import pandas as pd
import os

def heterodata_to_neo4j_csv(data, output_dir="neo4j_csv"):
    os.makedirs(output_dir, exist_ok=True)

    # Guardar nodos
    for node_type in data.node_types:
        node_data = data[node_type]

        # Asegurarse de que haya atributos
        if hasattr(node_data, 'x') and node_data.x is not None:
            df = pd.DataFrame(node_data.x.cpu().numpy())
            df.columns = [f"attr{i}" for i in range(df.shape[1])]
        else:
            num_nodes = data[node_type].num_nodes
            df = pd.DataFrame(index=range(num_nodes))  # Nodo sin atributos

        # Agregar columna de ID (requerida por Neo4j)
        df[f"{node_type}_id"] = df.index
        df.to_csv(os.path.join(output_dir, f"nodes_{node_type}.csv"), index=False)
        print(f"Guardado nodos: {node_type} -> {df.shape}")

    # Guardar aristas
    for edge_type in data.edge_types:
        src_type, rel_type, dst_type = edge_type
        edge_index = data[edge_type].edge_index.cpu().numpy()

        edge_df = pd.DataFrame({
            "source": edge_index[0],
            "target": edge_index[1],
            "type": rel_type,
            "source_type": src_type,
            "target_type": dst_type
        })

        # Agregar atributos de aristas si existen
        if 'edge_attr' in data[edge_type]:
            edge_attrs = data[edge_type]['edge_attr'].cpu().numpy()
            edge_attr_df = pd.DataFrame(edge_attrs)
            edge_df = pd.concat([edge_df, edge_attr_df], axis=1)

        filename = f"edges_{src_type}_{rel_type}_{dst_type}.csv"
        edge_df.to_csv(os.path.join(output_dir, filename), index=False)
        print(f"Guardado aristas: {edge_type} -> {edge_df.shape}")

    print("\n✅ Exportación completa. Los archivos están en:", os.path.abspath(output_dir))


# Llamar a la función
heterodata_to_neo4j_csv(data)

Guardado nodos: cuadra -> (4296, 9)
Guardado nodos: health_post -> (4, 3)
Guardado nodos: water_channel -> (218, 3)
Guardado aristas: ('cuadra', 'vecina_de', 'cuadra') -> (64086, 5)
Guardado aristas: ('cuadra', 'reporta_a', 'health_post') -> (4296, 6)
Guardado aristas: ('health_post', 'atiende_a', 'cuadra') -> (4296, 6)
Guardado aristas: ('cuadra', 'cerca_de', 'water_channel') -> (9228, 6)

✅ Exportación completa. Los archivos están en: /home/pantro/Documentos/GITHUB/rabies-graph-neural-model/neo4j_csv


# Armar datos para el random forest

In [ ]:
from shapely.ops import nearest_points

In [84]:
# TRAIN_VAL
rf_gdf_rabies = gdf_rabies_train_val.copy()

# Cambiar a "EPSG:32719"
rf_gdf_rabies_proj = rf_gdf_rabies.to_crs("EPSG:32719")
rf_gdf_economic_income_proj = gdf_economic_income.to_crs("EPSG:32719")
rf_gdf_health_posts_proj = gdf_health_posts.to_crs("EPSG:32719")
rf_gdf_water_channels_proj = gdf_water_channels.to_crs("EPSG:32719")

In [99]:
cuadras = rf_gdf_economic_income_proj.copy()

cuadras["centroid"] = cuadras.geometry.centroid

# One-hot encoding ingreso económico
cuadras = pd.get_dummies(cuadras, columns=["ingreso_economico"], prefix="ingreso")

# Casos de rabia dentro de la cuadra
cuadras["num_rabies_cases_in_cuadra"] = cuadras.geometry.apply(
    lambda poly: rabies[rabies.within(poly)].shape[0]
)


# Crear una lista con las geometrías de los puestos de salud (POINT)
health_points = rf_gdf_health_posts_proj.geometry.unary_union
# Calcular la distancia mínima a cada cuadra
cuadras['dist_to_nearest_health_post'] = cuadras.geometry.apply(
    lambda g: g.distance(nearest_points(g, health_points)[1])
)

# Parámetro: distancia máxima para considerar la torrenteras como relevantes
MAX_DIST = 500  # en metros
# Unimos las torrenteras en una sola geometría multiparte
torrenteras_geom = rf_gdf_water_channels_proj.geometry.unary_union
# Calcular distancia a torrenteras (si > MAX_DIST => 9999)
cuadras['dist_to_nearest_water_channel'] = cuadras.geometry.apply(
    lambda g: g.distance(torrenteras_geom) if g.distance(torrenteras_geom) <= MAX_DIST else 9999
)

CUADRA_PROXIMITY_RABIES_CASE_RADIUS = 100
# Distancia al caso de rabia más cercano en 100 m
def min_dist_to_case(poly_centroid, rabies_points, max_dist):
    distances = rabies_points.distance(poly_centroid)
    close = distances[distances <= max_dist]
    return close.min() if not close.empty else np.nan
cuadras["min_dist_to_rabies_case"] = cuadras["centroid"].apply(
    lambda p: min_dist_to_case(p, rabies.geometry, CUADRA_PROXIMITY_RABIES_CASE_RADIUS)
)
# Rellenar NaN con un valor alto para que redes neuronales lo manejen mejor
cuadras["min_dist_to_rabies_case"] = cuadras["min_dist_to_rabies_case"].fillna(9999)


# Creando etiqueta Y
cuadras['y'] = (cuadras['num_rabies_cases_in_cuadra'] > 0).astype(int)

print(cuadras['y'].value_counts())
cuadras.head()

y
0    4221
1      75
Name: count, dtype: int64


,cuadra_id,geometry,centroid,ingreso_B,ingreso_C,ingreso_D,ingreso_E,num_rabies_cases_in_cuadra,dist_to_nearest_health_post,dist_to_nearest_water_channel,min_dist_to_rabies_case,y
0,1.4.100-1,"POLYGON ((224268.545 8190480.422, 224175.196 8...",POINT (224132.601 8190627.741),False,False,True,False,0,1815.732788,9999.000000,9999.000000,0
1,1.4.100-10,"POLYGON ((223983.171 8191303.516, 223985.935 8...",POINT (223989.033 8191286.737),False,False,True,False,0,1610.846277,415.993967,9999.000000,0
2,1.4.100-11,"POLYGON ((223991.362 8191312.242, 224167.947 8...",POINT (224080.678 8191340.585),False,False,True,False,0,1787.481405,476.290826,32.510141,0
3,1.4.100-12,"POLYGON ((223736.505 8191347.126, 223726.379 8...",POINT (223784.996 8191332.717),False,False,True,False,0,1521.736024,339.202959,9999.000000,0
4,1.4.100-2,"POLYGON ((223940.104 8190652.198, 223924.309 8...",POINT (223949.842 8190607.880),False,False,True,False,0,1778.237966,9999.000000,9999.000000,0


In [101]:
# Eliminando centroide
cuadras.drop(columns=['centroid'], inplace=True)

cuadras.to_file('rf_cuadras.gpkg', driver='GPKG')